<a href="https://colab.research.google.com/github/Art-phys/Lesson_HF_LR/blob/main/Lesson_HF_RL_Unit5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Блок 5: Введение в ML-агенты

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/thumbnail.png" alt="Thumbnail" width="70%"/>

В этом блокноте вы узнаете о ML-агентах и обучите двух агентов.

- Первый из них научится **стрелять снежками по появляющимся мишеням**.
- Второму нужно нажать кнопку, чтобы создать пирамиду, затем перейти к пирамиде, опрокинуть ее **и перейти к золотому кирпичу наверху**. Чтобы сделать это, ему нужно будет исследовать свое окружение, и мы будем использовать технику, называемую любопытство (curiosity).

После этого вы сможете **наблюдать за игрой ваших агентов прямо в вашем браузере**.

⬇️ Вот пример того, чего **вы достигнете в конце этого блока.** ⬇️

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/pyramids.gif" alt="Pyramids"/>

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/snowballtarget.gif" alt="SnowballTarget" width="70%"/>

### 🎮 Окружающая среда: 

- Пирамиды [Pyramids](https://github.com/Unity-Technologies/ml-agents/blob/main/docs/Learning-Environment-Examples.md#pyramids)
- Мишень для снежков (SnowballTarget)

### 📚 RL-Library: 

- [ML-Агенты (Экспериментальная версия с HuggingFace)](https://github.com/huggingface/ml-agents)

⚠ Мы собираемся использовать экспериментальную версию ML-Agents, где вы можете использовать hub и загружать из hub модели Unity ML-Agents **вам необходимо установить ту же версию**

## Цели этго блокнота 🏆

В конце блокнота вы:

- Поймете, как работает **ML-Agents** и библиотека окружения.
- Научитесь **обучать агентов в средах Unity**.


<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/deep-rl-course-illustration.jpg" alt="Deep RL Course illustration" width="70%"/>

## Предварительные условия 🏗️
Прежде чем начать, вам нужно:

🔲  📚 ****Изучить [что такое ML-агенты и как это работает, прочитав блок 5](https://huggingface.co/deep-rl-course/unit5/introduction )** 🤗

# Давайте обучим наших агентов 🚀

Интеграция ML-агентов на хабе **все еще является экспериментальной**, некоторые функции будут добавлены в будущем.

Но на данный момент, **чтобы подтвердить это практическое руководство для процесса сертификации, вам просто нужно отправить ваши обученные модели в центр**. Нет никаких результатов, которых можно было бы достичь, чтобы подтвердить это. Но если вы хотите получить хорошие результаты, вы можете попытаться достичь:

- Для `Pyramids` : Среднюю награду= 1.75
- Для `SnowballTarget` : Среднюю награду = 15 или 30 целей, пораженных в эпизоде.

## Клонируйте репозиторий и установите зависимости 🔽
- Нам нужно клонировать репозиторий, который **содержит экспериментальную версию библиотеки, которая позволяет вам отправлять вашего обученного агента в концентратор.**

In [ ]:
%%capture
# Клонируйте репозиторий
!git clone --depth 1 https://github.com/huggingface/ml-agents/ 

In [ ]:
%%capture
# Зайдите в репозиторий и установите пакет
%cd ml-agents
!pip3 install -e ./ml-agents-envs
!pip3 install -e ./ml-agents

##Мишень для снежка ⛄

Если вам нужно уточнить, как работают эти среды, ознакомьтесь с этим разделом 👉
https://huggingface.co/deep-rl-course/unit5/snowball-target

### Загрузите и переместите zip-файл среды в `./training-envs-executables/linux/`
- Исполняемый файл нашей среды находится в zip-файле.
- Нам нужно загрузить его и поместить в `./training-envs-executables/linux/`
- Мы используем исполняемый файл linux, потому что мы используем colab, а ОС colab machines - Ubuntu (linux)

In [ ]:
# Здесь мы создаем папки training-envs-executables и linux
!mkdir ./training-envs-executables
!mkdir ./training-envs-executables/linux

Скачайте файл SnowballTarget.zip с https://drive.google.com/file/d/1YHHLjyj6gaZ3Gemx1hQgqrPgSS2ZhmB5 используя `wget`. 

Ознакомьтесь с полным решением для загрузки больших файлов из GDrive [здесь](https://bcrf.biochem.wisc.edu/2021/02/05/download-google-drive-files-using-wget/)

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YHHLjyj6gaZ3Gemx1hQgqrPgSS2ZhmB5' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1YHHLjyj6gaZ3Gemx1hQgqrPgSS2ZhmB5" -O ./training-envs-executables/linux/SnowballTarget.zip && rm -rf /tmp/cookies.txt

Распакуем файл executable.zip

In [ ]:
%%capture
!unzip -d ./training-envs-executables/linux/ ./training-envs-executables/linux/SnowballTarget.zip

Убедитесь, что ваш файл доступен

In [ ]:
!chmod -R 755 ./training-envs-executables/linux/SnowballTarget

### Настройте конфигурационный файл SnowballTarget
- Для ML-Agents вы определяете гиперпараметры обучения **в файлах config.yaml.**

Существует несколько гиперпараметров. Чтобы узнать их лучше, вы должны изучить [документацию](https://github.com/Unity-Technologies/ml-agents/blob/release_20_docs/docs/Training-Configuration-File.md)


Итак, вам нужно создать`SnowballTarget.yaml` конфигурационный файл в папке ./content/ml-agents/config/ppo/

Мы предоставим вам здесь первую версию этой конфигурации (для копирования и вставки в ваш файл `SnowballTarget.yaml`), **но вы должны изменить его**.

```
behaviors:
  SnowballTarget:
    trainer_type: ppo
    summary_freq: 10000
    keep_checkpoints: 10
    checkpoint_interval: 50000
    max_steps: 200000
    time_horizon: 64
    threaded: true
    hyperparameters:
      learning_rate: 0.0003
      learning_rate_schedule: linear
      batch_size: 128
      buffer_size: 2048
      beta: 0.005
      epsilon: 0.2
      lambd: 0.95
      num_epoch: 3
    network_settings:
      normalize: false
      hidden_units: 256
      num_layers: 2
      vis_encode_type: simple
    reward_signals:
      extrinsic:
        gamma: 0.99
        strength: 1.0
```

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/snowballfight_config1.png" alt="Config SnowballTarget" width="70%"/>
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit7/snowballfight_config2.png" alt="Config SnowballTarget" width="70%"/>